# Installs

In [1]:
%pip install pycryptodome
%pip install cryptography

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Hashing

## using pycryptodome

In [2]:
from Crypto.Hash import SHA3_256, BLAKE2s, RIPEMD160, keccak

msg = "Hello"
h1 = SHA3_256.new()
h2 = BLAKE2s.new()
h3 = RIPEMD160.new()
h4 = keccak.new(digest_bits=512)

h1.update(msg.encode())
h2.update(msg.encode())
h3.update(msg.encode())
h4.update(msg.encode())

print(f"SHA3-256   of '{msg}': {h1.hexdigest()}")
print(f"BLAKE2s    of '{msg}': {h2.hexdigest()}")
print(f"RIPEMD160  of '{msg}': {h3.hexdigest()}")
print(f"Keccak-512 of '{msg}': {h4.hexdigest()}")

SHA3-256   of 'Hello': 8ca66ee6b2fe4bb928a8e3cd2f508de4119c0895f22e011117e22cf9b13de7ef
BLAKE2s    of 'Hello': f73a5fbf881f89b814871f46e26ad3fa37cb2921c5e8561618639015b3ccbb71
RIPEMD160  of 'Hello': d44426aca8ae0a69cdbc4021c64fa5ad68ca32fe
Keccak-512 of 'Hello': c33fede18a1ae53ddb8663710f8054866beb714044fce759790459996196f101d94dfc7bd8268577f7ee3d2f8ff0cef4004a9632227db84df62d2b40682d69e2


## using Hashlib

In [3]:
import hashlib

msg = "Hello"

# print(hashlib.algorithms_available)
h1 = hashlib.new('sha3_512')
h2 = hashlib.new('blake2s')
h3 = hashlib.new('ripemd160')

h1.update(msg.encode())
h2.update(msg.encode())
h3.update(msg.encode())

print(f"SHA3-256   of '{msg}': {h1.hexdigest()}")
print(f"BLAKE2s    of '{msg}': {h2.hexdigest()}")
print(f"RIPEMD160  of '{msg}': {h3.hexdigest()}")

SHA3-256   of 'Hello': 0b8a44ac991e2b263e8623cfbeefc1cffe8c1c0de57b3e2bf1673b4f35e660e89abd18afb7ac93cf215eba36dd1af67698d6c9ca3fdaaf734ffc4bd5a8e34627
BLAKE2s    of 'Hello': f73a5fbf881f89b814871f46e26ad3fa37cb2921c5e8561618639015b3ccbb71
RIPEMD160  of 'Hello': d44426aca8ae0a69cdbc4021c64fa5ad68ca32fe


# Diffie-Hellman Key Exchange with sign

In [4]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh, dsa
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import serialization


In [5]:
param = dh.generate_parameters(generator=2, key_size=2048)
pvt1 = param.generate_private_key()
pub1 = pvt1.public_key()
pvt2 = param.generate_private_key()
pub2 = pvt2.public_key()
print(f"Pvt-Pub key pair for encryption generated for both parties")

Pvt-Pub key pair for encryption generated for both parties


In [6]:
sign_pvt1 = dsa.generate_private_key(key_size=2048)
sign_pub1 = sign_pvt1.public_key()
sign_pvt2 = dsa.generate_private_key(key_size=2048)
sign_pub2 = sign_pvt2.public_key()
print(f"Signing key generated for both parties")

Signing key generated for both parties


In [7]:
pub1_bytes = pub1.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)
pub2_bytes = pub2.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)
sign1 = sign_pvt1.sign(pub1_bytes, hashes.SHA3_256())
sign2 = sign_pvt2.sign(pub2_bytes, hashes.SHA3_256())
print(f"Signatures generated")

Signatures generated


In [8]:
sign_pub1.verify(
    signature=sign1,
    data=pub1_bytes,
    algorithm=hashes.SHA3_256(),
)
sign_pub2.verify(
    signature=sign2,
    data=pub2_bytes,
    algorithm=hashes.SHA3_256(),
)
print("Signatures verified")

Signatures verified


In [9]:
shared1 = pvt1.exchange(pub2)
shared2 = pvt2.exchange(pub1)
print(f" Shared secrets match: {shared1==shared2}")

 Shared secrets match: True


# Key Derivation

## PBKDF2

In [10]:
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Hash import SHA3_512
from Crypto.Random import get_random_bytes

password_ = "a secret"
salt_ = get_random_bytes(16)
key = PBKDF2(
    password=password_,
    salt=salt_,
    dkLen=64,
    count=1000000,
    hmac_hash_module=SHA3_512,
)
print(f"Password: {password_}")
print(f"Random salt: {salt_.hex()}")
print(f"Key: {key.hex()}")

Password: a secret
Random salt: 4e5a6f5ec739efb9614abff69763e69b
Key: 9624007fed51c5446e8a715cadb60975e13985c759c31866bed472a25e00c2a6815fa7a473104c5c41e9527cfddfa9976bab7d1e94c1a0413cc537af863e6216


## Scrypt

In [11]:
from Crypto.Protocol.KDF import scrypt
from Crypto.Random import get_random_bytes

password_ = "some secret"
salt_ = get_random_bytes(16)
key = scrypt(
    password=password_,
    salt=salt_,
    key_len=64,
    N=(1 << 10),
    r=8,
    p=1,
)
print(f"Password: {password_}")
print(f"Random salt: {salt_.hex()}")
print(f"Key: {key.hex()}")

Password: some secret
Random salt: 2a5f48529378f1dbfea91c2ecd35460f
Key: d32f1c3909b63165c0c70919c50a9dd0634a2c81c5cc576061a036bada0866b82af8f3e0651ec1f34ec9ba1502ce8f17abd9e2b29399a7d448efe5997306a0f9


# Encryption and Decryption

## AES-256-GCM

In [12]:
from Crypto.Cipher import AES
import hashlib

msg = "Message for aes-256-gcm encryption"
pwd = "something"
hasher = hashlib.new('sha3_256')
hasher.update(pwd.encode())
pwd_hash = hasher.digest()

aes_cipher = AES.new(pwd_hash, AES.MODE_GCM)
ct, auth_tag = aes_cipher.encrypt_and_digest(msg.encode())
iv = aes_cipher.nonce

aes_decipher = AES.new(pwd_hash, AES.MODE_GCM, iv)
pt = aes_decipher.decrypt(ct)

print(f"Cipher text: {ct.hex()}")
print(f"Auth Tag: {auth_tag.hex()}")
print(f"Nonce: {iv.hex()}")
print(f"Password: {pwd}")

try:
    aes_decipher.verify(auth_tag)
    print(f"Tag verified!")
    print(f"Plain text: {pt}")
except:
    print(f"Msg tampered!")
    print(f"Plain text: {pt}")


Cipher text: 10f4b44af10daaa092e7ec70bdd056acd1a337822a54e302eec4eac7c1cd11d48c15
Auth Tag: e6e0cd9bd250d7c6a3e7e499d7f410b6
Nonce: 1b4c2a003ec9e4714f160913a9c4eb4c
Password: something
Tag verified!
Plain text: b'Message for aes-256-gcm encryption'


## AES-256-CBC

In [13]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

msg = "Message for aes-256-cbc encryption"
pwd = "something"
hasher = hashlib.new("sha256")
hasher.update(pwd.encode())
pwd_hash = hasher.digest()

aes_cipher = AES.new(pwd_hash, AES.MODE_CBC)
ct = aes_cipher.encrypt(pad(msg.encode(), AES.block_size, style="pkcs7"))
iv = aes_cipher.iv

aes_decipher = AES.new(pwd_hash, AES.MODE_CBC, iv)
pt = unpad(aes_decipher.decrypt(ct), AES.block_size, "pkcs7")

print(f"Cipher text: {ct.hex()}")
print(f"IV: {iv.hex()}")
print(f"Password: {pwd}")
print(f"Pwd hash: {pwd_hash.hex()}")
print(f"Plain Text: {pt}")

Cipher text: 311bc41aad612354d8e1255e661ca625644a6bef032a66a5710ccf7904fc442bd672cbd0bf55d438a3304c939454ad7d
IV: 8226331234cc0e9fe949b6a0a2f54a31
Password: something
Pwd hash: 3fc9b689459d738f8c88a3a48aa9e33542016b7a4052e001aaa536fca74813cb
Plain Text: b'Message for aes-256-cbc encryption'


## AES-256-CTR

In [14]:
from Crypto.Cipher import AES

msg = "Message for aes-256-cbc encryption"
pwd = "something"
hasher = hashlib.new("sha3_256")
hasher.update(pwd.encode())
pwd_hash = hasher.digest()

aes_cipher = AES.new(pwd_hash, AES.MODE_CTR)
ct = aes_cipher.encrypt(msg.encode())
nonce = aes_cipher.nonce

aes_decipher = AES.new(pwd_hash, AES.MODE_CTR, nonce=nonce)
pt = aes_decipher.decrypt(ct)

print(f"Cipher text: {ct.hex()}")
print(f"IV: {nonce.hex()}")
print(f"Password: {pwd}")
print(f"Plain Text: {pt}")

Cipher text: 6c9d0e0ffd0ac820fd6b65cc6d1b8971513f2d4575d0deaccc2427ae11429435c4e8
IV: 041ea6949d1ec562
Password: something
Plain Text: b'Message for aes-256-cbc encryption'


## DES

In [15]:
# from Crypto.Cipher import DES3


# while True:
#     try:
#         key = DES3.adjust_key_parity(get_random_bytes(24))
#     except:
#         pass

# des_cipher = DES3.new(key, DES3.MODE_CBC)


## ChaCha20

In [16]:
from Crypto.Cipher import ChaCha20

msg = "Message for ChaCha20 encryption"
pwd = "something"
hasher = hashlib.new("sha3_256")
hasher.update(pwd.encode())
pwd_hash = hasher.digest()

chacha_cipher = ChaCha20.new(key=pwd_hash)
ct = chacha_cipher.encrypt(msg.encode())
nonce = chacha_cipher.nonce

chacha_decipher = ChaCha20.new(key=pwd_hash, nonce=chacha_cipher.nonce)
pt = chacha_decipher.decrypt(ct)

print(f"Cipher text: {ct.hex()}")
print(f"IV: {nonce.hex()}")
print(f"Password: {pwd}")
print(f"Plain Text: {pt}")

Cipher text: eb727d73702beb6ce255438b3c00073f7f8a98791b59e1906fa5d9b71baedf
IV: 8a166a4f9feb33a2
Password: something
Plain Text: b'Message for ChaCha20 encryption'


## RSA with PKCS1 OEAP

In [17]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

msg = "Message for ChaCha20 encryption"

key_pair = RSA.generate(1024)
pub_key = key_pair.public_key()
# print(pub_key.export_key())

n = key_pair.n
e = key_pair.e
d = key_pair.d

key_pair1 = RSA.construct((n, e, d))
key_pair2 = RSA.import_key(key_pair.export_key())
# print(f"Is key_pair == key pair_1: {key_pair==key_pair1}")
# print(f"Is key_pair == key pair_2: {key_pair==key_pair2}")

rsa_cipher = PKCS1_OAEP.new(key=pub_key)
ct = rsa_cipher.encrypt(msg.encode())

rsa_decipher = PKCS1_OAEP.new(key=key_pair)
pt = rsa_decipher.decrypt(ct)

print(f"Pub key: (n,e): ({pub_key.n},{pub_key.e})")
print(f"Pvt key: (n,d): ({key_pair.n},{key_pair.d})")
print(f"Cipher text: {ct.hex()}")
print(f"Plain text: {pt}")

Pub key: (n,e): (121891468033421275176841576625675901417688061675911861900935385724088184683240967476256378538917702221456990577045011239021218577430315529661225982105062543784590704132012730389451581961316257055552040108955868702597217049200425453435614751434526143805735098399838468645102143538028609922029889226473784792203,65537)
Pvt key: (n,d): (121891468033421275176841576625675901417688061675911861900935385724088184683240967476256378538917702221456990577045011239021218577430315529661225982105062543784590704132012730389451581961316257055552040108955868702597217049200425453435614751434526143805735098399838468645102143538028609922029889226473784792203,50538731722113558911459422336229782706300833115942948615806601405017749399540821356368380823626512084221902207150832207728818414094541463409745542391944457336213641861721447508098874565188267472366552121459858660112519982402120192439358938481002484187561712442304535969525957583731698388701547028397977005425)
Cipher text: 41b3ba9f3044f560

## ECC with AES-256-GCM (hybrid)

In [18]:
from Crypto.PublicKey import ECC
from Crypto.PublicKey.ECC import EccPoint
from Crypto.Hash import SHA3_256
from Crypto.Cipher import AES

# key_pair = ECC.generate(curve="p521")
# pub_key = key_pair.public_key()
pvt1 = ECC.generate(curve="p521")
pvt2 = ECC.generate(curve="p521")
pub1 = pvt1.public_key()
pub2 = pvt2.public_key()

shared1 = pub2.pointQ * pvt1.d
shared2 = pub1.pointQ * pvt2.d
assert shared1 == shared2


def get_key(a: EccPoint):
    hasher = SHA3_256.new(a.x.to_bytes())
    hasher.update(a.y.to_bytes())
    return hasher.hexdigest()

msg = "I am ironman"
key = get_key(shared1)

aes_cipher = AES.new(bytes.fromhex(key), AES.MODE_GCM)
ct, auth_tag = aes_cipher.encrypt_and_digest(msg.encode())
nonce_ = aes_cipher.nonce

aes_decipher = AES.new(bytes.fromhex(key), AES.MODE_GCM, nonce=nonce_)
pt = aes_decipher.decrypt(ct)

try:
    aes_decipher.verify(auth_tag)
    print(f"Msg verified: {pt}")
except:
    print(f"Msg not verified: {pt}")


Msg verified: b'I am ironman'


# Signatures

## with RSA, PKCS1 v1.5

In [19]:
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA3_256
from Crypto.PublicKey import RSA

key = RSA.generate(1024)
msg = "I am inevitable"

pwd = b"something"
with open("rsa_key.pem", "wb") as f:
    data = key.export_key(
        passphrase=pwd,
        pkcs=8,
        protection="PBKDF2WithHMAC-SHA512AndAES256-CBC",
        prot_params={"iteration_count": 131072},
    )
    f.write(data)

with open("rsa_key.pem", "rb") as f:
    data = f.read()
    key_pair = RSA.import_key(data, pwd)

# assert key_pair == key

# Sender's side
pub_key = key_pair.public_key()
signer = PKCS1_v1_5.new(key_pair)
sign = signer.sign(SHA3_256.new(msg.encode()))

# Receiver's side
validator = PKCS1_v1_5.new(pub_key)
val = validator.verify(SHA3_256.new(msg.encode()), sign)
print(f"Signature validity: {val}")

Signature validity: True


## with ECC, DSS

In [20]:
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA3_256
from Crypto.Signature import DSS

ecc_key = ECC.generate(curve="p256")

pwd = "something"
with open("ecc_key.pem", "wt") as f:
    data = ecc_key.export_key(
        format="PEM",
        passphrase=pwd.encode(),
        protection="PBKDF2WithHMAC-SHA512AndAES256-CBC",
        prot_params={"iteration_count": 131072},
    )
    f.write(data)
with open("ecc_key.pem", "rt") as f:
    data = f.read()
    key_pair = ECC.import_key(data, pwd.encode())

msg = "I am inevitable"

# Sender's side
pub_key = key_pair.public_key()
signer = DSS.new(key=key_pair, mode="fips-186-3")
sign = signer.sign(SHA3_256.new(msg.encode()))

# Receiver's side
verifier = DSS.new(key=pub_key, mode='fips-186-3')
try:
    verifier.verify(SHA3_256.new(msg.encode()), sign)
    print("Signature is valid!")
except:
    print("Invalid Sign")

Signature is valid!
